In [1]:
import cv2
import numpy as np

def extract_frames(video_path, fps=2):
    cap = cv2.VideoCapture(video_path)
    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    target_fps = min(frame_rate, fps)
    
    frames = []
    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        if frame_count % int(frame_rate / target_fps) == 0:
            frames.append(frame)
        frame_count += 1
    
    cap.release()
    return frames

def compute_farneback_optical_flow(frames):
    prev_gray = cv2.cvtColor(frames[0], cv2.COLOR_BGR2GRAY)
    flow_maps = []
    
    # Initialize Farneback optical flow parameters
    params = dict(
        pyr_scale=0.5,
        levels=3,
        winsize=15,
        iterations=3,
        poly_n=5,
        poly_sigma=1.2,
        flags=0
    )

    for frame in frames[1:]:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        flow_map = cv2.calcOpticalFlowFarneback(prev_gray, gray, flow=None, **params)
        flow_maps.append(flow_map)
        prev_gray = gray
    return flow_maps

def downscale_and_average_flow_maps(flow_maps):
    downscaled_maps = [
        cv2.resize(
            flow, (16, int(flow.shape[0] * (16 / flow.shape[1]))), interpolation=cv2.INTER_AREA
        ) 
        for flow in flow_maps
    ]
    average_flow_map = np.mean(np.array(downscaled_maps), axis=0)
    return np.mean(average_flow_map)

In [2]:
video_path = "v_BasketballDunk_g14_c06-Scene-002.mp4"
frames = extract_frames(video_path)
flow_maps = compute_farneback_optical_flow(frames)
motion_score = downscale_and_average_flow_maps(flow_maps)

In [3]:
motion_threshold = 0.1

# Filter out static scenes based on the threshold
is_static_scene = motion_score < motion_threshold

print("Motion Score:", motion_score)
print("Is Static Scene:", is_static_scene)

Motion Score: -1.2653071
Is Static Scene: True
